<a href="https://colab.research.google.com/github/JimKing100/DS-Unit-2-Applied-Modeling/blob/master/DS_Unit_2_Study_Guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [134]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Applied-Modeling.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module4')

Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Applied-Modeling
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
import pandas as pd

# Stratified sample, 10% of expired Lending Club loans, grades A-D
# Source: https://www.lendingclub.com/info/download-data.action
history = pd.read_csv('../data/lending-club/lending-club-subset.csv')
history['issue_d'] = pd.to_datetime(history['issue_d'], infer_datetime_format=True)

# Current loans available for manual investing, June 17, 2019
# Source: https://www.lendingclub.com/browse/browse.action
current = pd.read_csv('../data/lending-club/primaryMarketNotes_browseNotes_1-RETAIL.csv')

In [0]:
# Transform earliest_cr_line to an integer:
# How many days the earliest credit line was open, before the loan was issued.
# For current loans available for manual investing, assume the loan will be issued today.
history['earliest_cr_line'] = pd.to_datetime(history['earliest_cr_line'], infer_datetime_format=True)
history['earliest_cr_line'] = history['issue_d'] - history['earliest_cr_line']
history['earliest_cr_line'] = history['earliest_cr_line'].dt.days

current['earliest_cr_line'] = pd.to_datetime(current['earliest_cr_line'], infer_datetime_format=True)
current['earliest_cr_line'] = pd.Timestamp.today() - current['earliest_cr_line']
current['earliest_cr_line'] = current['earliest_cr_line'].dt.days

# Transform earliest_cr_line for the secondary applicant
history['sec_app_earliest_cr_line'] = pd.to_datetime(history['sec_app_earliest_cr_line'], infer_datetime_format=True, errors='coerce')
history['sec_app_earliest_cr_line'] = history['issue_d'] - history['sec_app_earliest_cr_line']
history['sec_app_earliest_cr_line'] = history['sec_app_earliest_cr_line'].dt.days

current['sec_app_earliest_cr_line'] = pd.to_datetime(current['sec_app_earliest_cr_line'], infer_datetime_format=True, errors='coerce')
current['sec_app_earliest_cr_line'] = pd.Timestamp.today() - current['sec_app_earliest_cr_line']
current['sec_app_earliest_cr_line'] = current['sec_app_earliest_cr_line'].dt.days

# Engineer features for issue date year & month
history['issue_d_year'] = history['issue_d'].dt.year
history['issue_d_month'] = history['issue_d'].dt.month

current['issue_d_year'] = pd.Timestamp.today().year
current['issue_d_month'] = pd.Timestamp.today().month

In [0]:
# Calculate percent of each loan repaid
history['percent_paid'] = history['total_pymnt'] / history['funded_amnt']

In [138]:
history.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,...,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,issue_d_year,issue_d_month,percent_paid
0,109889419.0,NaN,20000.0,20000.0,20000.0,36 months,9.44%,640.10,B,B1,IT Support,10+ years,MORTGAGE,99500.0,Source Verified,2017-05-01,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,296xx,SC,17.19,0.0,4472,690.0,694.0,0.0,NaN,NaN,15.0,0.0,12835.0,41%,28.0,w,0.0,0.0,...,28300.0,84078.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,2017,5,1.082883
1,68579794.0,NaN,21600.0,21600.0,21600.0,36 months,9.80%,694.95,B,B3,Engineer Tech,10+ years,MORTGAGE,79000.0,Verified,2016-01-01,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,219xx,MD,18.94,1.0,7519,680.0,684.0,0.0,12.0,NaN,13.0,0.0,17498.0,88.4%,16.0,w,0.0,0.0,...,10100.0,38591.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,2016,1,1.156630
2,10105952.0,NaN,25000.0,25000.0,24950.0,36 months,17.57%,898.43,D,D2,Retail manager,7 years,MORTGAGE,90100.0,Source Verified,2013-12-01,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/13 > credit card and...,home_improvement,home,017xx,MA,19.58,1.0,7215,695.0,699.0,0.0,0.0,NaN,15.0,0.0,12440.0,49%,36.0,f,0.0,0.0,...,14000.0,61185.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,2013,12,1.293732
3,57713137.0,NaN,5000.0,5000.0,5000.0,36 months,6.89%,154.14,A,A3,Business Analyst,3 years,MORTGAGE,110139.0,Verified,2015-08-01,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,641xx,MO,33.01,0.0,6148,715.0,719.0,0.0,NaN,NaN,19.0,0.0,29415.0,84.8%,52.0,w,0.0,0.0,...,13900.0,138487.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,2015,8,1.098508
4,11966074.0,NaN,11850.0,11850.0,11850.0,36 months,14.16%,405.93,C,C2,Account Supervisor,6 years,RENT,50500.0,Source Verified,2014-02-01,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 02/18/14 > I'm trying to g...,credit_card,Credit card refinancing,373xx,TN,20.82,0.0,3959,670.0,674.0,2.0,69.0,NaN,9.0,0.0,6774.0,61.6%,15.0,f,0.0,0.0,...,4700.0,48771.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,2014,2,1.144327


In [139]:
current.head()

,id,member_id,loan_amnt,funded_amnt,term,int_rate,exp_default_rate,service_fee_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,is_inc_v,accept_d,exp_d,list_d,credit_pull_d,review_status_d,review_status,url,desc,purpose,title,zip_code,addr_state,msa,acc_now_delinq,acc_open_past_24mths,bc_open_to_buy,percent_bc_gt_75,bc_util,dti,delinq_2yrs,delinq_amnt,earliest_cr_line,fico_range_low,fico_range_high,...,num_rev_tl_bal_gt_0,num_op_rev_tl,tot_coll_amt,effective_int_rate,application_type,disbursement_method,annual_inc_joint,dti_joint,verified_status_joint,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,inq_fi,total_cu_tl,inq_last_12m,mtg_payment,housing_payment,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,issue_d_year,issue_d_month
0,152167636,186565679,22000.0,19675.0,60,11.71,5.49,0.73,486.16,B,B3,NaN,< 1 year,MORTGAGE,60000.0,Not Verified,05-24-2019 06:08:38,07-07-2019 14:00:00,06-07-2019 14:00:00,05-21-2019 09:55:17,06-07-2019 10:00:50,Approved,https://lendingclub.com/browse/loanDetail.acti...,,Credit card refinancing,Credit card refinancing,751xx,TX,NaN,0,2,13573,40.0,58.2,14.65,0,0.0,7234,665,669,...,5,8,989,11.60,INDIVIDUAL,DIRECT_PAY,,,,2,0,0,0,73,0.0,,2,2,9285.0,54.6,0,3,2,736.00,,,,,NaN,,,,,,,,,,2019,10
1,153003150,188095322,28000.0,26725.0,36,8.19,2.16,0.86,879.88,A,A4,NaN,< 1 year,RENT,80000.0,Not Verified,06-06-2019 21:39:53,07-07-2019 18:00:00,06-07-2019 18:00:00,06-04-2019 20:38:53,06-07-2019 13:56:56,Approved,https://lendingclub.com/browse/loanDetail.acti...,,Debt consolidation,Debt consolidation,921xx,CA,NaN,0,1,13197,75.0,71.0,32.19,0,0.0,5149,715,719,...,7,8,0,8.18,INDIVIDUAL,CASH,,,,0,6,0,1,16,103666.0,58.7,0,0,7843.0,63.0,1,2,0,0.00,,,,,NaN,,,,,,,,,,2019,10
2,152654206,187356227,27000.0,22500.0,36,10.33,5.15,1.01,875.41,B,B1,Logistics Manager,1 year,RENT,72000.0,Source Verified,06-05-2019 07:21:02,07-06-2019 10:00:00,06-06-2019 10:00:00,05-30-2019 04:47:10,06-06-2019 06:02:41,Approved,https://lendingclub.com/browse/loanDetail.acti...,,Credit card refinancing,Credit card refinancing,480xx,MI,NaN,0,4,5880,40.0,75.0,21.38,0,0.0,4608,675,679,...,4,8,151,10.22,INDIVIDUAL,DIRECT_PAY,,,,1,2,1,2,7,13471.0,55.7,1,2,11381.0,55.2,1,1,2,0.00,,,,,NaN,,,,,,,,,,2019,10
3,152209289,186627083,40000.0,15725.0,36,25.00,12.86,1.42,1590.40,D,D4,NaN,3 years,MORTGAGE,9500.0,Verified,05-27-2019 08:52:04,07-08-2019 14:00:00,06-08-2019 14:00:00,05-21-2019 15:03:03,06-08-2019 09:57:43,Approved,https://lendingclub.com/browse/loanDetail.acti...,,Debt consolidation,Debt consolidation,421xx,KY,NaN,0,2,922,100.0,92.6,175.22,0,0.0,5072,670,674,...,7,7,0,24.20,JOINT,CASH,109500.0,28.66,Verified,0,2,1,1,9,48458.0,91.7,0,1,6203.0,89.4,6,0,12,867.79,,60345,655,659,5072.0,4,3,16,78.0,3,15,0,0,,2019,10
4,151779963,185691648,32000.0,20100.0,60,12.40,5.49,0.73,718.31,B,B4,Correction officer,10+ years,RENT,65000.0,Source Verified,06-03-2019 10:54:50,07-13-2019 10:00:00,06-13-2019 10:00:00,05-14-2019 13:50:30,06-13-2019 06:01:31,Approved,https://lendingclub.com/browse/loanDetail.acti...,,Credit card refinancing,Credit card refinancing,112xx,NY,NaN,0,7,13541,0.0,16.9,6.09,0,0.0,1700,715,719,...,4,7,0,12.29,INDIVIDUAL,CASH,,,,0,1,1,1,8,8541.0,85.4,3,6,1400.0,41.0,0,2,0,0.00,,,,,NaN,,,,,,,,,,2019,10


In [0]:
# Train on the historical data.
# For the target, use `loan_status` ('Fully Paid' or 'Charged Off')
target = 'loan_status'
X = history.drop(columns=target)
y = history[target]

In [141]:
# Do train/validate/test 3-way split
from sklearn.model_selection import train_test_split

# Split into training/validation and test
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=20000, stratify=y, random_state=42)

# Split training/validation into training and validation
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=20000, 
    stratify=y_trainval, random_state=42)

print('X_train shape', X_train.shape)
print('y_train shape', y_train.shape)
print('X_val shape', X_val.shape)
print('y_val shape', y_val.shape)
print('X_test shape', X_test.shape)
print('y_test shape', y_test.shape)

X_train shape (88334, 152)
y_train shape (88334,)
X_val shape (20000, 152)
y_val shape (20000,)
X_test shape (20000, 152)
y_test shape (20000,)


In [0]:
# Save the ids for later, so we can look up actual results,
# to compare with predicted results
train_id = X_train['id']
val_id = X_val['id']
test_id = X_test['id']

In [0]:
# Use Python sets to compare the historical columns & current columns
common_columns = set(history.columns) & set(current.columns)
just_history = set(history.columns) - set(current.columns)
just_current = set(current.columns) - set(history.columns)

In [0]:
# For features, use only the common columns shared by the historical & current data.
features = list(common_columns)
X_train = X_train[features]
X_val = X_val[features]
X_test = X_test[features]

In [145]:
X_test.shape

(20000, 108)

In [146]:
def wrangle(X):
    X = X.copy()

    # Engineer new feature for every feature: is the feature null?
    for col in X:
        X[col+'_NULL'] = X[col].isnull()
    
    # Convert percentages from strings to floats
    X['int_rate'] = X['int_rate'].str.strip('%').astype(float)
    X['revol_util'] = X['revol_util'].str.strip('%').astype(float)
    
    # Convert employment length from string to float
    X['emp_length'] = X['emp_length'].str.replace(r'\D','').astype(float)
        
    # Create features for three employee titles: teacher, manager, owner
    X['emp_title'] = X['emp_title'].str.lower()
    X['emp_title_teacher'] = X['emp_title'].str.contains('teacher', na=False)
    X['emp_title_manager'] = X['emp_title'].str.contains('manager', na=False)
    X['emp_title_owner']   = X['emp_title'].str.contains('owner', na=False)

    # Get length of free text fields
    X['title'] = X['title'].str.len()
    X['desc'] = X['desc'].str.len()
    X['emp_title'] = X['emp_title'].str.len()
    
    # Convert sub_grade from string "A1"-"D5" to numbers
    sub_grade_ranks = {'A1': 1.1, 'A2': 1.2, 'A3': 1.3, 'A4': 1.4, 'A5': 1.5, 
                       'B1': 2.1, 'B2': 2.2, 'B3': 2.3, 'B4': 2.4, 'B5': 2.5, 
                       'C1': 3.1, 'C2': 3.2, 'C3': 3.3, 'C4': 3.4, 'C5': 3.5, 
                       'D1': 4.1, 'D2': 4.2, 'D3': 4.3, 'D4': 4.4, 'D5': 4.5}
    X['sub_grade'] = X['sub_grade'].map(sub_grade_ranks)
    
    # Drop some columns
    X = X.drop(columns='id')        # Always unique
    X = X.drop(columns='url')       # Always unique
    X = X.drop(columns='member_id') # Always null
    X = X.drop(columns='grade')     # Duplicative of sub_grade
    X = X.drop(columns='zip_code')  # High cardinality
    
    # Only use these features which had nonzero permutation importances in earlier models    
    features = ['acc_open_past_24mths', 'addr_state', 'all_util', 'annual_inc', 
                'annual_inc_joint', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 
                'collections_12_mths_ex_med', 'delinq_amnt', 'desc_NULL', 'dti', 
                'dti_joint', 'earliest_cr_line', 'emp_length', 'emp_length_NULL', 
                'emp_title', 'emp_title_NULL', 'emp_title_owner', 'fico_range_high', 
                'funded_amnt', 'home_ownership', 'inq_last_12m', 'inq_last_6mths', 
                'installment', 'int_rate', 'issue_d_month', 'issue_d_year', 'loan_amnt', 
                'max_bal_bc', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 
                'mo_sin_rcnt_rev_tl_op', 'mort_acc', 'mths_since_last_major_derog_NULL', 
                'mths_since_last_record', 'mths_since_recent_bc', 'mths_since_recent_inq', 
                'num_actv_bc_tl', 'num_actv_rev_tl', 'num_op_rev_tl', 'num_rev_tl_bal_gt_0', 
                'num_tl_120dpd_2m_NULL', 'open_rv_12m_NULL', 'open_rv_24m', 
                'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec_bankruptcies', 'purpose', 
                'revol_bal', 'revol_bal_joint', 'sec_app_earliest_cr_line', 
                'sec_app_fico_range_high', 'sec_app_open_acc', 'sec_app_open_act_il', 
                'sub_grade', 'term', 'title', 'title_NULL', 'tot_coll_amt', 
                'tot_hi_cred_lim', 'total_acc', 'total_bal_il', 'total_bc_limit', 
                'total_cu_tl', 'total_rev_hi_lim']    
    X = X[features]
    
    # Reset index
    X = X.reset_index(drop=True)
    
    # Return the wrangled dataframe
    return X


X_train = wrangle(X_train)
X_val   = wrangle(X_val)
X_test  = wrangle(X_test)

print('X_train shape', X_train.shape)
print('X_val shape', X_val.shape)
print('X_test shape', X_test.shape)

X_train shape (88334, 66)
X_val shape (20000, 66)
X_test shape (20000, 66)


In [150]:
X_val.head()

,acc_open_past_24mths,addr_state,all_util,annual_inc,annual_inc_joint,avg_cur_bal,bc_open_to_buy,bc_util,collections_12_mths_ex_med,delinq_amnt,desc_NULL,dti,dti_joint,earliest_cr_line,emp_length,emp_length_NULL,emp_title,emp_title_NULL,emp_title_owner,fico_range_high,funded_amnt,home_ownership,inq_last_12m,inq_last_6mths,installment,int_rate,issue_d_month,issue_d_year,loan_amnt,max_bal_bc,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mort_acc,mths_since_last_major_derog_NULL,mths_since_last_record,mths_since_recent_bc,mths_since_recent_inq,num_actv_bc_tl,num_actv_rev_tl,num_op_rev_tl,num_rev_tl_bal_gt_0,num_tl_120dpd_2m_NULL,open_rv_12m_NULL,open_rv_24m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,purpose,revol_bal,revol_bal_joint,sec_app_earliest_cr_line,sec_app_fico_range_high,sec_app_open_acc,sec_app_open_act_il,sub_grade,term,title,title_NULL,tot_coll_amt,tot_hi_cred_lim,total_acc,total_bal_il,total_bc_limit,total_cu_tl,total_rev_hi_lim
0,7.0,NH,NaN,111000.0,NaN,21089.0,26385.0,45.5,0.0,0.0,True,18.90,NaN,6667,7.0,False,7.0,False,False,714.0,23475.0,OWN,NaN,1.0,583.29,16.99,1,2014,23475.0,NaN,125.0,219.0,1.0,8.0,True,NaN,1.0,5.0,6.0,10.0,26.0,10.0,False,True,NaN,96.8,30.0,0.0,debt_consolidation,35808.0,NaN,NaN,NaN,NaN,NaN,4.1,60 months,18.0,False,0.0,606784.0,62.0,NaN,48400.0,NaN,81000.0
1,6.0,FL,63.0,65000.0,NaN,21849.0,8703.0,11.2,0.0,0.0,True,19.42,NaN,1948,1.0,False,5.0,False,False,679.0,5000.0,MORTGAGE,2.0,0.0,185.82,20.00,1,2018,5000.0,723.0,60.0,64.0,1.0,1.0,True,NaN,22.0,8.0,2.0,5.0,8.0,5.0,False,False,4.0,93.3,50.0,0.0,other,11008.0,NaN,NaN,NaN,NaN,NaN,4.4,36 months,5.0,False,0.0,290232.0,16.0,84408.0,9800.0,4.0,24300.0
2,NaN,NY,NaN,65000.0,NaN,NaN,NaN,NaN,0.0,0.0,True,24.15,NaN,3287,4.0,False,22.0,False,False,679.0,21250.0,RENT,NaN,1.0,746.57,15.95,8,2010,21250.0,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,NaN,NaN,NaN,0.0,debt_consolidation,0.0,NaN,NaN,NaN,NaN,NaN,4.4,36 months,18.0,False,NaN,NaN,24.0,NaN,NaN,NaN,NaN
3,4.0,NE,47.0,53000.0,NaN,10756.0,14310.0,34.7,0.0,0.0,True,17.32,NaN,20910,10.0,False,18.0,False,False,784.0,13200.0,MORTGAGE,0.0,0.0,298.59,12.74,4,2017,13200.0,6839.0,125.0,687.0,8.0,4.0,True,NaN,8.0,NaN,2.0,5.0,9.0,5.0,False,False,2.0,100.0,0.0,0.0,debt_consolidation,7829.0,NaN,NaN,NaN,NaN,NaN,3.1,60 months,18.0,False,0.0,159347.0,26.0,21236.0,21900.0,0.0,33300.0
4,1.0,IN,33.0,67700.0,NaN,6414.0,25194.0,30.0,0.0,0.0,True,25.24,NaN,6758,3.0,False,18.0,False,False,789.0,8800.0,RENT,0.0,0.0,265.02,5.32,1,2016,8800.0,6505.0,169.0,222.0,7.0,0.0,True,NaN,7.0,NaN,2.0,2.0,9.0,2.0,False,False,1.0,100.0,0.0,0.0,debt_consolidation,10806.0,NaN,NaN,NaN,NaN,NaN,1.1,36 months,18.0,False,0.0,138246.0,32.0,66161.0,36000.0,2.0,42000.0


In [147]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import shap
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import eli5
from eli5.sklearn import PermutationImportance
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from pdpbox.pdp import pdp_isolate, pdp_plot

processor = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median')
)

X_train_processed = processor.fit_transform(X_train)
X_val_processed = processor.transform(X_val)
X_val_df = pd.DataFrame(X_val_processed, columns=)

eval_set = [(X_train_processed, y_train), 
            (X_val_processed, y_val)]

model = XGBClassifier(n_estimators=1000, n_jobs=-1)
model.fit(X_train_processed, y_train, eval_set=eval_set, eval_metric='auc', 
          early_stopping_rounds=10)

[0]	validation_0-auc:0.669638	validation_1-auc:0.672822
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.670802	validation_1-auc:0.674118
[2]	validation_0-auc:0.676142	validation_1-auc:0.680087
[3]	validation_0-auc:0.679771	validation_1-auc:0.683186
[4]	validation_0-auc:0.682482	validation_1-auc:0.684256
[5]	validation_0-auc:0.683905	validation_1-auc:0.685868
[6]	validation_0-auc:0.685104	validation_1-auc:0.687372
[7]	validation_0-auc:0.685857	validation_1-auc:0.687737
[8]	validation_0-auc:0.685805	validation_1-auc:0.687888
[9]	validation_0-auc:0.686504	validation_1-auc:0.689168
[10]	validation_0-auc:0.686866	validation_1-auc:0.689461
[11]	validation_0-auc:0.686938	validation_1-auc:0.689288
[12]	validation_0-auc:0.687537	validation_1-auc:0.69022
[13]	validation_0-auc:0.688371	validation_1-auc:0.690954
[14]	validation_0-auc:0.688811	validation_1-auc:0.691599

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=1000, n_jobs=-1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [157]:
X_val_processed

array([[7.00e+00, 4.50e+01, 5.90e+01, ..., 4.84e+04, 0.00e+00, 8.10e+04],
       [6.00e+00, 4.00e+00, 6.30e+01, ..., 9.80e+03, 4.00e+00, 2.43e+04],
       [4.00e+00, 7.00e+00, 5.90e+01, ..., 1.56e+04, 0.00e+00, 2.47e+04],
       ...,
       [5.00e+00, 7.00e+00, 4.30e+01, ..., 4.75e+04, 0.00e+00, 4.75e+04],
       [9.00e+00, 8.00e+00, 7.50e+01, ..., 2.90e+04, 2.00e+00, 4.71e+04],
       [6.00e+00, 3.90e+01, 5.90e+01, ..., 1.85e+04, 0.00e+00, 1.99e+04]])

In [149]:
X_val_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65
0,7.0,45.0,59.0,111000.0,107000.0,21089.0,26385.0,45.5,0.0,0.0,1.0,18.90,18.02,6667.0,7.0,0.0,7.0,0.0,0.0,714.0,23475.0,1.0,2.0,1.0,583.29,16.99,1.0,2014.0,23475.0,4242.0,125.0,219.0,1.0,8.0,1.0,72.0,1.0,5.0,6.0,10.0,26.0,10.0,0.0,1.0,2.0,96.8,30.0,0.0,1.0,35808.0,24640.5,5022.0,674.0,10.0,2.0,4.1,2.0,18.0,0.0,0.0,606784.0,62.0,23524.0,48400.0,0.0,81000.0
1,6.0,4.0,63.0,65000.0,107000.0,21849.0,8703.0,11.2,0.0,0.0,1.0,19.42,18.02,1948.0,1.0,0.0,5.0,0.0,0.0,679.0,5000.0,2.0,2.0,0.0,185.82,20.00,1.0,2018.0,5000.0,723.0,60.0,64.0,1.0,1.0,1.0,72.0,22.0,8.0,2.0,5.0,8.0,5.0,0.0,0.0,4.0,93.3,50.0,0.0,4.0,11008.0,24640.5,5022.0,674.0,10.0,2.0,4.4,1.0,5.0,0.0,0.0,290232.0,16.0,84408.0,9800.0,4.0,24300.0
2,4.0,7.0,59.0,65000.0,107000.0,7589.0,5101.0,61.6,0.0,0.0,1.0,24.15,18.02,3287.0,4.0,0.0,22.0,0.0,0.0,679.0,21250.0,3.0,2.0,1.0,746.57,15.95,8.0,2010.0,21250.0,4242.0,130.0,166.0,8.0,1.0,1.0,72.0,14.0,5.0,3.0,5.0,7.0,5.0,1.0,1.0,2.0,98.0,40.0,0.0,1.0,0.0,24640.5,5022.0,674.0,10.0,2.0,4.4,1.0,18.0,0.0,0.0,116107.5,24.0,23524.0,15600.0,0.0,24700.0
3,4.0,44.0,47.0,53000.0,107000.0,10756.0,14310.0,34.7,0.0,0.0,1.0,17.32,18.02,20910.0,10.0,0.0,18.0,0.0,0.0,784.0,13200.0,2.0,0.0,0.0,298.59,12.74,4.0,2017.0,13200.0,6839.0,125.0,687.0,8.0,4.0,1.0,72.0,8.0,5.0,2.0,5.0,9.0,5.0,0.0,0.0,2.0,100.0,0.0,0.0,1.0,7829.0,24640.5,5022.0,674.0,10.0,2.0,3.1,2.0,18.0,0.0,0.0,159347.0,26.0,21236.0,21900.0,0.0,33300.0
4,1.0,15.0,33.0,67700.0,107000.0,6414.0,25194.0,30.0,0.0,0.0,1.0,25.24,18.02,6758.0,3.0,0.0,18.0,0.0,0.0,789.0,8800.0,3.0,0.0,0.0,265.02,5.32,1.0,2016.0,8800.0,6505.0,169.0,222.0,7.0,0.0,1.0,72.0,7.0,5.0,2.0,2.0,9.0,2.0,0.0,0.0,1.0,100.0,0.0,0.0,1.0,10806.0,24640.5,5022.0,674.0,10.0,2.0,1.1,1.0,18.0,0.0,0.0,138246.0,32.0,66161.0,36000.0,2.0,42000.0


In [155]:
X_val_df.columns

RangeIndex(start=0, stop=66, step=1)

In [156]:
feature = 12

isolated = pdp_isolate(
    model = model,
    dataset = X_val_df,
    model_features=X_val_df.columns,
    feature=feature
)

pdp_plot(isolated, feature_name=feature);

ValueError: ignored

In [0]:
# Plot ROC curve
fpr, tpr, thresholds = roc_curve(y_val=='Fully Paid', y_pred_proba)
plt.plot(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate');

In [0]:
permuter = PermutationImportance(
    model, 
    scoring='accuracy',
    n_iter=2,
    random_state=42
)

permuter.fit(X_val_processed, y_val)
feature_names = X_val.columns.tolist()

eli5.show_weights(
    permuter,
    top=None,
    feature_names = feature_names
)

In [0]:
majority_class = y_train.mode()[0]
majority_class

In [0]:
y_pred = np.full_like(y_val, fill_value=majority_class)
y_pred

In [0]:
accuracy_score(y_val, y_pred)

In [0]:
print(classification_report(y_val, y_pred))

In [0]:
X_test_processed = processor.transform(X_test)
tester = model.predict_proba(X_test_processed)
tester

In [0]:
y_pred_proba = model.predict_proba(X_test_processed)[:, 1]
y_pred_proba

In [0]:
print('Test ROC AUC:', roc_auc_score(y_test, y_pred_proba))

In [0]:
X_val.head()

In [0]:
X_val.columns

In [0]:
# feature = 'dti'

# isolated = pdp_isolate(
#     model = model,
#     dataset = X_val,
#     model_features=X_val.columns,
#     feature=feature
# )

# pdp_plot(isolated, feature_name=feature);

In [0]:
X_test.iloc[[3094]]

In [0]:
row = X_test.iloc[[3094]]

explainer = shap.TreeExplainer(model)
row_process = processor.transform(row)
shap_values = explainer.shap_values(row_process)

shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value,
    shap_values=shap_values,
    features=row
)